In [171]:
import time
import pprint
import re
import numpy as np

from openeye import oechem
from openeye import oeomega
import qcportal as ptl

# Conforming molecules

In [172]:
# Custom exception for the case when there is no nitrogen
class NoNitrogenException(Exception): pass

def find_nitrogen(mol):
    """Returns the trivalent nitrogen atom in a molecule"""
    for atom in mol.GetAtoms():
        if oechem.OEIsInvertibleNitrogen()(atom):
            return atom, atom.GetIdx()
    raise NoNitrogenException()

In [173]:
# Initialize Omega
omega = oeomega.OEOmega()

omega.SetMaxConfs(1)
omega.SetIncludeInput(True)
omega.SetCanonOrder(True)
omega.SetSampleHydrogens(True)  # Word to the wise: skipping this step can lead to significantly different charges!
omega.SetStrictStereo(True)
omega.SetStrictAtomTypes(True)
omega.SetIncludeInput(False) # don't include input

# QM Calculations

In [174]:
client = ptl.FractalClient("https://localhost:7777/", verify=False)
print(client)

FractalClient(server_name='QCFractal Server', address='https://localhost:7777/', username='None')


In [175]:
def make_ptl_mol(oemol):
    """Builds a QCPortal Molecule from an OpenEye molecule"""
    coords = oemol.GetCoords()
    coord_str = '\n'.join(
        (f"{oechem.OEGetAtomicSymbol(atom.GetAtomicNum())}   "
                   f"{'   '.join(str(c) for c in coords[atom.GetIdx()])}")
                  for atom in mol.GetAtoms())
    print(coord_str)
    conn = np.array([[bond.GetBgnIdx(), bond.GetEndIdx(), bond.GetOrder()] for bond 
            in mol.GetBonds()])
    return ptl.Molecule.from_data(coord_str, connectivity=conn)

In [178]:
def send_qm_job(ptl_mol, nitrogen, nitrogen_i):
    """Sends a job to the QM Client - returns a submitted object"""
    indices = [nitrogen_i] + [nbor.GetIdx() for nbor in list(nitrogen.GetAtoms())]
    print(f"indices: {indices}")
    keywords = ptl.models.KeywordSet(values={"scf_properties":["wiberg_lowdin_indices"]})
    keywords_id = client.add_keywords([keywords])[0]
    service = ptl.models.GridOptimizationInput(**{
            "keywords": {
                "preoptimization": True,
                "scans": [{
                    "type": "dihedral",
                    "indices": indices,
                    "steps": [-40, -36, -32, -28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
                    "step_type": "absolute"
                }]
            },
            "optimization_spec": {
                "program": "geometric",
                "keywords": {
                    "coordsys": "tric",
                }
            },
            "qc_spec": {
                "driver": "gradient",
                "method": "mp2",
                "basis": "def2-SV(P)",
                "keywords": keywords_id,
                "program": "psi4",
                
#                 "driver": "gradient",
#                 "method": "UFF",
#                 "basis": None,
#                 "keywords": None,
#                 "program": "rdkit",
            },
            "initial_molecule": ptl_mol,
        })
    submitted = client.add_service([service])
    print("Job submitted")
    return submitted

# Final Run

In [179]:
#This is where we submit the job. 
#The molecule we ran this example with stored as a smile string in the tiny.smi file. 
#This should be adapted for the directory "Molecules_to_run" for the .sdf files

tmp_mol = oechem.OEMol()
ifs = oechem.oemolistream("tiny.smi")
first = True

results = [] # {"molecule": <OEMol>, "nitrogen": <OEAtom>, "nitrogen_i": <int>,
             #  "ptl_molecule": <PtlMol>, submitted": <submitted object>,
             #  "res": <result object> from QCPortal}

while oechem.OEReadMolecule(ifs, tmp_mol):
    # Separate outputs by a line
    if first: first = False
    else: print()
    
    mol = oechem.OEMol(tmp_mol)
    status = omega(mol)
    nitrogen, nitrogen_i = find_nitrogen(mol)
    print(f"Nitrogen found at index {nitrogen_i}")
    print(f"Generating conformer: {'done' if status else 'failed'}")
    
    print(f"Nitrogen is at: {mol.GetCoords()[nitrogen_i]}")
    
    ptl_mol = make_ptl_mol(mol)
    sub = send_qm_job(ptl_mol, nitrogen, nitrogen_i)
    
    results.append({
        "molecule": mol,
        "nitrogen": nitrogen,
        "nitrogen_i": nitrogen_i,
        "ptl_molecule": ptl_mol,
        "submitted": sub,
    })
    break

print()
pprint.PrettyPrinter(indent = 2).pprint(results)

Nitrogen found at index 2
Generating conformer: done
Nitrogen is at: (1.0301599502563477, 0.2824336886405945, 1.800162672996521)
C   1.0277451276779175   0.13483351469039917   0.3510288894176483
C   2.385664463043213   0.4585181176662445   2.3034117221832275
N   1.0301599502563477   0.2824336886405945   1.800162672996521
H   1.6256577968597412   -0.7329955101013184   0.06077375262975693
H   1.449762225151062   1.026540994644165   -0.12041513621807098
H   0.006210207939147949   -0.003914475440979004   -0.013064242899417877
H   3.0003867149353027   -0.4053081274032593   2.037337064743042
H   2.374415874481201   0.5605388283729553   3.3919730186462402
H   2.835350751876831   1.3567941188812256   1.8718671798706055
H   0.4692816138267517   1.0947539806365967   2.0555851459503174
indices: [2, 9, 0, 1]
Job submitted

[ { 'molecule': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x1234acdb0> >,
    'nitrogen': <oechem.OEAtomBase; proxy of <Swig Object of type 'OEChem::OEAto

In [180]:
# Query for an update on the jobs every 5 seconds
total_jobs = len(results)
completed = 0
while completed != total_jobs:
    print("Running check")
    for i in range(total_jobs):
        if "res" in results[i]: continue # if "res" has already been set then the job has already finished
        res = client.query_procedures(id=results[i]["submitted"].ids)[0]
        
        print(i, ":", res)
        
        # this is a very sketchy way for checking if the job is done - find a better way!
        if str(res.status) != "RecordStatusEnum.incomplete":
            print(f"Calculation {i} finished")
            results[i]["res"] = res
            completed += 1
    time.sleep(5)

Running check
0 : GridOptimizationRecord(id='5d0402af45144f1df5c9b558' status='COMPLETE')
Calculation 0 finished


In [181]:
# Print final results
pprint.PrettyPrinter(indent = 2).pprint(results)

[ { 'molecule': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x1234acdb0> >,
    'nitrogen': <oechem.OEAtomBase; proxy of <Swig Object of type 'OEChem::OEAtomBase *' at 0x1238b2870> >,
    'nitrogen_i': 2,
    'ptl_molecule': <Molecule(name='C2H7N' formula='C2H7N' hash='8629b41')>,
    'res': <GridOptimizationRecord(id='5d0402af45144f1df5c9b558' status='COMPLETE')>,
    'submitted': <ComputeResponse(nsubmitted=0 nexisting=1)>}]


In [183]:
#gridopt = client.query_procedures({"id": results[0]['res']})[0]
gridopt = client.query_procedures(id= results[0]['res'].id)[0]
